<a href="https://colab.research.google.com/github/OJB-Quantum/MuMax3-How-To/blob/main/Python%20Code_MuMax3%20Data%20Plots/SOT_Effect_Simulation_with_MuMax3_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Check GPU + driver
!nvidia-smi --query-gpu="name,driver_version,compute_cap" --format=csv

name, driver_version, compute_cap
NVIDIA L4, 550.54.15, 8.9


In [2]:
#@title Install MuMax³ (MuMax³ 3.10 CUDA 10.1)
# Download the mumax3 binary
!wget -q https://mumax.ugent.be/mumax3-binaries/mumax3.10_linux_cuda10.1.tar.gz
!tar -xvf mumax3.10_linux_cuda10.1.tar.gz
!rm mumax3.10_linux_cuda10.1.tar.gz
!rm -rf mumax3.10 && mv mumax3.10_linux_cuda10.1 mumax3.10

# Update the PATH environment variable
import os
os.environ["PATH"] += ":/content/mumax3.10"

mumax3.10_linux_cuda10.1/
mumax3.10_linux_cuda10.1/mumax3-server
mumax3.10_linux_cuda10.1/lib/
mumax3.10_linux_cuda10.1/lib/libcurand.so.10
mumax3.10_linux_cuda10.1/lib/libcufft.so.10
mumax3.10_linux_cuda10.1/mumax3
mumax3.10_linux_cuda10.1/LICENSE
mumax3.10_linux_cuda10.1/mumax3-convert


In [3]:
# @title Step 2 — Write SOT demo (.mx3) with control knobs (PEP 8/257)
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from textwrap import dedent


@dataclass
class Knobs:
    """User controls for a MuMax³ SOT pulse demo.

    Geometry: PMA CoFeB-like nanodot on a heavy metal underlayer (Pt-like).
    SOT is injected as two custom effective fields:
    B_DL ∝ m × σ  and  B_FL ∝ σ.
    """
    # Grid / geometry (nm)
    nx: int = 256
    ny: int = 256
    nz: int = 1
    dx_nm: float = 2.0
    dy_nm: float = 2.0
    t_nm: float = 1.2
    use_dot: bool = True
    dot_diam_nm: float = 160.0  # Circle() expects DIAMETER

    # Material (SI)
    ms_a_per_m: float = 1.10e6
    aex_j_per_m: float = 1.50e-11
    ku1_j_per_m3: float = 6.0e5
    alpha: float = 0.02

    # SOT (spin Hall driven)
    theta_sh: float = 0.10        # Pt-like spin Hall angle
    xi_fl_over_dl: float = 0.60   # H_FL / H_DL ratio

    # Spin polarization σ (unit vector) for +x current, +z normal → σ ≈ +y
    p_x: float = 0.0
    p_y: float = 1.0
    p_z: float = 0.0

    # Drive: SOT pulse, then relax
    jc_on_a_per_m2: float = 2.0e11
    pulse_ns: float = 1.0
    relax_ns: float = 1.0

    # Output cadence
    table_dt_ps: float = 5.0
    snap_dt_ps: float = 50.0

    # Temperature (0 K deterministic; set > 0 for thermal noise)
    temp_k: float = 0.0

    # Files
    script_path: Path = Path("/content/sot_demo.mx3")
    out_dir: Path = Path("/content/sot_out")


def write_mx3(k: Knobs) -> Path:
    """Write a MuMax³ input file that applies a SOT pulse via custom fields."""
    csx = k.dx_nm * 1e-9
    csy = k.dy_nm * 1e-9
    csz = k.t_nm * 1e-9
    diam = k.dot_diam_nm * 1e-9   # Circle() takes diameter (m)
    t_ps = k.table_dt_ps * 1e-12
    s_ps = k.snap_dt_ps * 1e-12
    run1 = k.pulse_ns * 1e-9
    run2 = k.relax_ns * 1e-9

    geom = f"SetGeom(Circle({diam}))" if k.use_dot else "// full rectangle"

    # NOTE: Use ConstVector(...) for p so Cross(m, p) is a Quantity.
    # See workshop example and API "Custom effective field terms". :contentReference[oaicite:1]{index=1}
    mx3 = f"""
// ============================ GRID & GEOMETRY ===============================
SetGridSize({k.nx}, {k.ny}, {k.nz})
SetCellSize({csx}, {csy}, {csz})
SetPBC(0, 0, 0)
{geom}

// ================================ MATERIAL =================================
Msat  = {k.ms_a_per_m}
Aex   = {k.aex_j_per_m}
Ku1   = {k.ku1_j_per_m3}
AnisU = Vector(0, 0, 1)
alpha = {k.alpha}
{"Temp = %g" % k.temp_k if k.temp_k > 0 else ""}

// Initialize up (PMA) and relax slightly
m = Uniform(0, 0, 1)
Minimize()

// =============================== OUTPUT SCHEDULE ============================
AutoSave(m, {s_ps})
TableAutoSave({t_ps})

// =============================== PRE-PULSE RELAX ============================
Run(0.2e-9)

// =============================== SOT FIELDS (PULSE) ========================
// B_DL = a_J * (m × p),  B_FL = b_J * p
// a_J = (ħ * theta_SH / (2 e Ms t_F)) * Jc,  b_J = xi * a_J

theta_SH := {k.theta_sh}
xiSOT    := {k.xi_fl_over_dl}

e    := 1.602176634e-19
hbar := 1.054571817e-34
tF   := {csz}
Ms   := {k.ms_a_per_m}

p := ConstVector({k.p_x}, {k.p_y}, {k.p_z})   // <-- Quantity, not plain data.Vector
Jc := Const({k.jc_on_a_per_m2})

// Prefactor c [Tesla per (A/m^2)]
c  := Const(hbar * theta_SH / (2*e*Ms*tF))

// a_J and b_J (ScalarFunctions)
aJ := Mul(Jc, c)
bJ := Mul(aJ, Const(xiSOT))

// Custom effective fields promoted into B_eff
B_DL := Mul(aJ, Cross(m, p))   // → torque ∝ m × (σ × m)
B_FL := Mul(bJ, p)             // → torque ∝ m × σ

AddFieldTerm(B_DL)
AddFieldTerm(B_FL)

// Pulse on
Run({run1})

// ============================ REMOVE SOT & RELAX ============================
RemoveCustomFields()
Run({run2})

// Final snapshot
Save(m)
"""
    k.script_path.write_text(dedent(mx3).strip() + "\n", encoding="utf-8")
    return k.script_path


KNOBS = Knobs()
mx3_path = write_mx3(KNOBS)
print(f"Wrote: {mx3_path}")
print(f"Output dir: {KNOBS.out_dir}")


Wrote: /content/sot_demo.mx3
Output dir: /content/sot_out


In [ ]:
# @title Step 3 — Run MuMax³ with explicit output directory
from pathlib import Path

Path("/content/sot_out").mkdir(parents=True, exist_ok=True)
!mumax3 -o /content/sot_out -f /content/sot_demo.mx3

# Quick listing for sanity
!ls -lh /content/sot_out | sed -n '1,80p'


//mumax 3.10 [linux_amd64 go1.14(gc) CUDA-10.1]
//GPU info: NVIDIA L4(22692MB), CUDA Driver 12.4, cc=8.9, using cc=75 PTX
//(c) Arne Vansteenkiste, Dynamat LAB, Ghent University, Belgium
//This is free software without any warranty. See license.txt
//********************************************************************//
//  If you use mumax in any work or publication,                      //
//  we kindly ask you to cite the references in references.bib        //
//********************************************************************//
//output directory: /content/sot_out/
//starting GUI at http://127.0.0.1:35367
SetGridSize(256, 256, 1)
SetCellSize(2e-09, 2e-09, 1.2e-09)
SetPBC(0, 0, 0)
//resizing...
// Initializing geometry 0 %
// Initializing geometry 100 %
SetGeom(Circle(1.6e-07))
Msat = 1100000.0
Aex = 1.5e-11
Ku1 = 600000.0
AnisU = Vector(0, 0, 1)
alpha = 0.02
m = Uniform(0, 0, 1)
Minimize()
//Did not use cached kernel: open /tmp/mumax3kernel_[256 256 1]_[0 0 0]_[2e-09 2e-09 1.2e-

In [ ]:
# @title Step 4 — Read table and plot <m_z>(t) at 200 dpi (PEP 8/257)
from __future__ import annotations

from pathlib import Path
from typing import List
import pandas as pd
import matplotlib.pyplot as plt


def load_mumax_table(path: Path) -> pd.DataFrame:
    """Return DataFrame from a MuMax³ table file, renaming first 4 columns."""
    if not path.exists():
        raise FileNotFoundError(f"Missing MuMax³ table: {path}")
    df = pd.read_csv(path, sep=r"\s+", engine="python", comment="#", header=None)
    ncols = df.shape[1]
    names: List[str] = ["t", "mx", "my", "mz"] + [f"col{i}" for i in range(4, ncols)]
    df.columns = names[:ncols]
    return df


df = load_mumax_table(Path("/content/sot_out/table.txt"))
print(df.tail(8).to_string(index=False))

plt.rcParams.update({"figure.dpi": 200})
plt.figure()
plt.plot(df["t"], df["mz"], linewidth=1.8, color='blue')
plt.xlabel("time t [s]")
plt.ylabel("<m_z> [unitless]")
plt.title("SOT pulse on PMA CoFeB dot — average m_z(t)")
plt.grid(True)
plt.show()


In [ ]:
# @title Step 5 — Convert BEFORE & AFTER OVFs → PNG and show side-by-side
from __future__ import annotations

import subprocess
from pathlib import Path
from typing import Tuple, List
import glob

from PIL import Image
import matplotlib.pyplot as plt


OUT_DIR = Path("/content/sot_out")
COMPONENT: str = "z"   # "x", "y", "z" or 0, 1, 2 (per mumax3-convert manual)
ARROWS: int = 0        # set > 0 to overlay arrows; 0 disables arrows


def find_first_last_ovf(pattern: str = "m*.ovf") -> Tuple[Path, Path]:
    """Return paths to the first and last OVF magnetization snapshots."""
    files: List[str] = sorted(glob.glob(str(OUT_DIR / pattern)))
    if not files:
        raise FileNotFoundError(f"No OVF files matching {pattern} found in {OUT_DIR}")
    return Path(files[0]), Path(files[-1])


def convert_ovf_to_png(ovf: Path) -> Path:
    """Convert an OVF file to PNG via mumax3-convert and return the PNG path."""
    cmd = ["mumax3-convert", f"-comp={COMPONENT}", "-png", "-o", str(OUT_DIR), str(ovf)]
    if ARROWS:
        cmd.insert(1, f"-arrows={ARROWS}")
    subprocess.run(cmd, check=True)

    # The tool writes <stem>.png in OUT_DIR (or a very similar name).
    png_candidate = OUT_DIR / f"{ovf.stem}.png"
    if png_candidate.exists():
        return png_candidate

    matches = sorted(OUT_DIR.glob(f"{ovf.stem}*.png"))
    if not matches:
        raise FileNotFoundError(f"PNG not found after conversion for {ovf.name}")
    return matches[-1]


before_ovf, after_ovf = find_first_last_ovf()
before_png = convert_ovf_to_png(before_ovf)
after_png = convert_ovf_to_png(after_ovf)

plt.rcParams.update({"figure.dpi": 200})

fig, axes = plt.subplots(1, 2, figsize=(10, 4), constrained_layout=True)
axes[0].imshow(Image.open(before_png))
axes[0].set_title(f"Before (first: {before_ovf.name})")
axes[0].axis("off")

axes[1].imshow(Image.open(after_png))
axes[1].set_title(f"After (last: {after_ovf.name})")
axes[1].axis("off")

plt.show()

In [ ]:
# @title Step 5c — 2D quiver + up/down markers (BEFORE & AFTER) — 200 dpi
from __future__ import annotations

from pathlib import Path
from typing import List, Tuple, Optional
import re
import glob
import numpy as np
import matplotlib.pyplot as plt

# Install discretisedfield if needed
try:
    import discretisedfield as df  # type: ignore
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "discretisedfield"])
    import discretisedfield as df  # type: ignore


# ----------------------------- USER CONTROLS ---------------------------------
OUT_DIR: Path = Path("/content/sot_out")
MX3_PATH: Path = Path("/content/sot_demo.mx3")

Z_INDEX: int = 0                 # layer to visualize (0 for thin films)
SUBSAMPLE: int = 4               # plot every Nth arrow in x and y
ARROW_SCALE_FACTOR: float = 1.2  # unit-vector arrow length ≈ factor × mean cell size
DRAW_CIRCLE_OUTLINE: bool = True
MASK_OUTSIDE_CIRCLE: bool = True

# Styling
DPI: int = 200
TITLE_FS: int = 10
LABEL_FS: int = 9
TICK_FS: int = 8
LABEL_PAD: float = 3.0
TITLE_PAD: float = 4.0
ARROW_WIDTH: float = 0.006       # quiver shaft width (axes fraction)
DOT_SIZE: float = 10.0           # marker size for m_z >= 0
X_SIZE: float = 28.0             # marker size for m_z < 0 (cross)
UP_COLOR: str = "black"
DOWN_COLOR: str = "crimson"
ARROW_COLOR: str = "C0"


# ------------------------------- HELPERS -------------------------------------
def find_first_last_ovf(out_dir: Path) -> Tuple[Path, Path]:
    """Return first and last m*.ovf snapshots."""
    files = sorted(out_dir.glob("m*.ovf"))
    if not files:
        raise FileNotFoundError(f"No OVF files found in {out_dir}")
    return files[0], files[-1]


def parse_circle_diameter(mx3_path: Path) -> Optional[float]:
    """Parse SetGeom(Circle(diameter_in_meters)) if present."""
    if not mx3_path.exists():
        return None
    txt = mx3_path.read_text(encoding="utf-8", errors="ignore")
    m = re.search(r"SetGeom\s*\(\s*Circle\s*\(\s*([0-9eE.+-]+)\s*\)\s*\)", txt)
    return float(m.group(1)) if m else None


def load_layer_xy_and_m(path: Path, z_index: int) -> Tuple[np.ndarray, np.ndarray,
                                                           np.ndarray, np.ndarray, np.ndarray]:
    """Load one OVF and return X, Y, mx, my, mz on a chosen z-slice (cell centers)."""
    field = df.Field.from_file(str(path))   # OVF 1/2 autodetected
    nx, ny, nz = map(int, field.mesh.n)
    if not (0 <= z_index < nz):
        raise IndexError(f"Z_INDEX {z_index} out of range [0, {nz-1}]")
    dx, dy, dz = field.mesh.cell
    pmin = np.asarray(field.mesh.region.pmin, dtype=float)

    xs = pmin[0] + dx * (np.arange(nx) + 0.5)
    ys = pmin[1] + dy * (np.arange(ny) + 0.5)
    X, Y = np.meshgrid(xs, ys, indexing="ij")

    arr = field.array  # shape (nx, ny, nz, 3)
    mx = arr[..., 0][:, :, z_index]
    my = arr[..., 1][:, :, z_index]
    mz = arr[..., 2][:, :, z_index]
    return X, Y, mx, my, mz


def prepare_quiver_data(X: np.ndarray, Y: np.ndarray,
                        mx: np.ndarray, my: np.ndarray, mz: np.ndarray,
                        subsample: int,
                        radius: Optional[float],
                        mask_outside: bool) -> Tuple[np.ndarray, ...]:
    """Subsample, apply optional circular mask, and return arrays for plotting."""
    Xs = X[::subsample, ::subsample]
    Ys = Y[::subsample, ::subsample]
    mxs = mx[::subsample, ::subsample]
    mys = my[::subsample, ::subsample]
    mzs = mz[::subsample, ::subsample]

    if radius is not None and mask_outside:
        mask = (Xs**2 + Ys**2) <= (radius**2)
        mxs = np.where(mask, mxs, np.nan)
        mys = np.where(mask, mys, np.nan)
        mzs = np.where(mask, mzs, np.nan)

    return Xs, Ys, mxs, mys, mzs


def draw_quiver_panel(ax: plt.Axes,
                      X: np.ndarray, Y: np.ndarray,
                      mx: np.ndarray, my: np.ndarray, mz: np.ndarray,
                      title: str,
                      arrow_len_xy: float,
                      circle_diam: Optional[float]) -> None:
    """Plot in-plane quiver and overlay dot/X according to m_z sign."""
    # Compute arrow field scaled in XY units so arrows are clearly visible
    U = mx * arrow_len_xy
    V = my * arrow_len_xy

    # Quiver (keep arrows legible)
    q = ax.quiver(
        X, Y, U, V,
        angles="xy", scale_units="xy", scale=1.0,
        width=ARROW_WIDTH, headwidth=3.0, headlength=5.0,
        color=ARROW_COLOR
    )

    # Up vs down markers from m_z sign (NaNs ignored automatically)
    up = np.where(mz >= 0.0)
    dn = np.where(mz < 0.0)

    ax.scatter(X[up], Y[up], s=DOT_SIZE, marker=".", c=UP_COLOR, linewidths=0)
    ax.scatter(X[dn], Y[dn], s=X_SIZE, marker="x", c=DOWN_COLOR, linewidths=0.9)

    # Circle outline for context
    if circle_diam is not None and DRAW_CIRCLE_OUTLINE:
        r = circle_diam / 2.0
        t = np.linspace(0.0, 2.0 * np.pi, 360)
        ax.plot(r * np.cos(t), r * np.sin(t), "k-", linewidth=1.0)

    ax.set_aspect("equal", adjustable="box")
    ax.set_title(title, fontsize=TITLE_FS, pad=TITLE_PAD)
    ax.set_xlabel("x [m]", fontsize=LABEL_FS, labelpad=LABEL_PAD)
    ax.set_ylabel("y [m]", fontsize=LABEL_FS, labelpad=LABEL_PAD)
    ax.tick_params(axis="both", labelsize=TICK_FS)


# --------------------------------- MAIN --------------------------------------
before_path, after_path = find_first_last_ovf(OUT_DIR)
circle_diam = parse_circle_diameter(MX3_PATH)

# Load first & last frames
Xb, Yb, mxb, myb, mzb = load_layer_xy_and_m(before_path, Z_INDEX)
Xa, Ya, mxa, mya, mza = load_layer_xy_and_m(after_path, Z_INDEX)

# Subsample and mask (optional)
R = circle_diam / 2.0 if circle_diam is not None else None
Xb, Yb, mxb, myb, mzb = prepare_quiver_data(Xb, Yb, mxb, myb, mzb, SUBSAMPLE, R, MASK_OUTSIDE_CIRCLE)
Xa, Ya, mxa, mya, mza = prepare_quiver_data(Xa, Ya, mxa, mya, mza, SUBSAMPLE, R, MASK_OUTSIDE_CIRCLE)

# Arrow length from mean in-plane cell size (so arrows are in real XY units)
mean_dx = float(np.mean(np.diff(np.unique(Xb[:, 0])))) if Xb.shape[0] > 1 else 1.0
mean_dy = float(np.mean(np.diff(np.unique(Yb[0, :])))) if Yb.shape[1] > 1 else 1.0
arrow_len = ARROW_SCALE_FACTOR * float(np.mean([mean_dx, mean_dy]))

plt.rcParams.update({"figure.dpi": DPI})
fig, axes = plt.subplots(1, 2, figsize=(10.6, 4.6), constrained_layout=True)

draw_quiver_panel(
    axes[0], Xb, Yb, mxb, myb, mzb,
    title=f"Before — {before_path.name}",
    arrow_len_xy=arrow_len,
    circle_diam=circle_diam
)

draw_quiver_panel(
    axes[1], Xa, Ya, mxa, mya, mza,
    title=f"After — {after_path.name}",
    arrow_len_xy=arrow_len,
    circle_diam=circle_diam
)

plt.show()
